In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app

In [3]:
modeller = msm.convert('alanine_dipeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, constraints=app.HBonds, nonbondedMethod=app.NoCutoff)

In [4]:
explorer = oe.Explorer(topology, system, platform='CUDA')

In [5]:
explorer.set_coordinates(positions)

In [6]:
temperature = 1000.0*unit.kelvin
timestep = 2.0*unit.femtoseconds
collision_rate = 1.0/unit.picoseconds
explorer.md.langevin.set_parameters(temperature=temperature, timestep=timestep,
                                    collision_rate=collision_rate)

time_langevin_total = 10*unit.nanosecond
time_langevin_cicle = 10.0*unit.picoseconds
steps_langevin_cicle = int(time_langevin_cicle / explorer.md.langevin._timestep)
n_cicles = int(time_langevin_total/time_langevin_cicle)

minima = []

coordinates = explorer.get_coordinates()
explorer.quench.l_bfgs()
minima.append(explorer.get_coordinates())

explorer.set_coordinates(coordinates)
explorer.set_velocities_to_temperature(temperature)

threshold = 0.25*unit.angstroms

for cicle_index in range(n_cicles):
    
    explorer.md.langevin(steps_langevin_cicle)
    coordinates = explorer.get_coordinates()
    velocities = explorer.get_velocities()
    explorer.quench.l_bfgs()
    new_minimum=True
    for minimum in reversed(minima):
        distance = explorer.distance.least_rmsd(minimum)
        if distance<=threshold:
            new_minimum=False
            break
    if new_minimum:
        minima.append(explorer.get_coordinates())
    explorer.set_coordinates(coordinates)
    explorer.set_velocities(velocities)
    

In [7]:
len(minima)

3

Podemos calcular la distancia entre configuraciones para hacer un mapa por proximidad: mds o red con threshold.